In [55]:
my_set = {1,2,3}
if 1 

TypeError: unsupported operand type(s) for +: 'set' and 'tuple'

SyntaxError: incomplete input (515937610.py, line 1)

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.option_context('mode.use_inf_as_na', True)

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

# Preprocessing (Check shape, missing and duplicate values):

In [ ]:
path = r"C:\Users\kunbo\OneDrive\1. Học tập\Unigap - DAC\Machine Learning Module\Project\Final\final project\churn_prediction.xlsx"
df = pd.read_excel(path)

## Check first 5 rows:

In [ ]:
df.head()

## Check last 5 rows:

In [ ]:
df.tail()

## Check dataset information:

In [ ]:
df.info()

## Check imbalance data

In [ ]:
distribute = df.groupby('Churn', as_index=False)['Churn'].count()
distribute['%'] = distribute['Churn'] / len(df) * 100
distribute

- Tỉ lệ được phân bổ tốt, không cần thực hiện resample

## Check duplicated values

In [ ]:
df.duplicated().sum()

## Count null values and solve it if have:

In [ ]:
null_count=df.isna().sum().sum()
print(f'% Missing values vs total observations is: {null_count}/{len(df)} ~ {null_count / len(df)*100:.2f}%')

In [ ]:
df.isna().sum().sort_values(ascending=False)

\
**_Only Numeric columns have null values_**

### Fill null values with interpolate with _linear_ method

In [ ]:
new_df = df.interpolate()
new_df.isna().sum().sum()

In [ ]:
new_df.head()

### Separate String and Numeric columns

In [ ]:
cat_cols = []
num_cols = []

for col in new_df.columns:
    if new_df[col].dtype == 'object':
        cat_cols.append(col)
    else:
        num_cols.append(col)

In [ ]:
print(f'Type of this varible is: {type(cat_cols)} \n {cat_cols}')
print(f'Type of this varible is: {type(num_cols)} \n {num_cols}')

## Import Machine Learning Library

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score, KFold, train_test_split

### Covert String datatype to binary

In [ ]:
dummies_df = pd.get_dummies(new_df, dtype='int64')
dummies_df

### Separate Independent and Dependent Features

In [ ]:
X = dummies_df.drop('Churn', axis=1)
y = dummies_df.Churn

print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Scale Numeric Data to same scale

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
clf = DecisionTreeClassifier(max_depth=16, random_state=42)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

### Using Decision Tree model too pick important features for EDA process and modeling after:

In [ ]:
# Get feature importances
importances = clf.feature_importances_
 
# Select features with importance greater than a threshold
threshold = 0.06  # Adjust as needed
selected_features = X.columns[importances > threshold]
 
# Use only the selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

X_train_selected.head()

In [ ]:
importances

In [ ]:
X_train_scaled.shape
y_train.shape

# Start EDA by features picked from above process:

In [ ]:
important_features = ['Tenure', 'DaySinceLastOrder', 'CashbackAmount', 'Churn']

eda_df = new_df[important_features].copy()

In [ ]:
eda_df.groupby('Churn').agg({'Tenure': 'mean', 'DaySinceLastOrder': 'mean', 'CashbackAmount':'mean'})

## Correlation:

In [ ]:
ax = sns.heatmap(eda_df.corr(), annot=True)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

- Correlation cho thấy những khách hàng có thời gian gắn bó càng dài, Recency cao và CashBackAmout lớn sẽ có tỉ lệ rời bỏ thấp hơn

## Visualize each variables

In [ ]:
ax = eda_df['Tenure'].value_counts() \
                .head(10) \
                .plot(kind='bar', title='Top 10 Tenure')
ax.set_ylabel('Count')
plt.show()

In [ ]:
ax = eda_df['DaySinceLastOrder'].plot(kind='hist', bins=30, title='Recency')
ax.set_xlabel('Day Since Last Order')
plt.show()

In [ ]:
ax = eda_df['CashbackAmount'].plot(kind='hist', bins=20, title='Cashback Amout Distribute')
ax.set_xlabel('Cashback Amount')
plt.show()

### Churn rate by Tenure

In [ ]:
churn_rate = eda_df.query('Churn == 1').groupby(['Tenure'], as_index=False)['Churn'].count().sort_values(by='Tenure')

churn_rate['perc'] = (churn_rate['Churn'] / churn_rate['Churn'].sum() * 100).round(2)

# ax = churn_rate.query('perc > 1.75').loc[:, ['Tenure', 'perc']].set_index('Tenure').plot(kind='bar', title='')
ax = churn_rate.loc[:, ['Tenure', 'perc']].sort_values(by='perc', ascending=False).head(10).set_index('Tenure').plot(kind='bar', title='Top 10 tenure with highest churn rate')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.bar_label(container=ax.containers[0])
ax.set_ylabel('%')

plt.show()

In [ ]:
ax = eda_df.query('Churn == 1') \
           .groupby(['Tenure'], as_index=False)['Churn'] \
           .count().sort_values(by='Tenure') \
           .query('Churn > 10') \
           .set_index('Tenure') \
           .plot(kind='bar', title='Churn distribute by Tenure')

ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.bar_label(container=ax.containers[0])
ax.set_ylabel('Count')
plt.show()

## Relationship between variables:

In [ ]:
eda_df['Tenure'] = eda_df['Tenure'].round(2)

In [ ]:
ax = sns.scatterplot(x='Tenure', y='CashbackAmount', data=eda_df)
plt.show()

In [ ]:
sns.scatterplot(x='Tenure', y='DaySinceLastOrder', data=eda_df)
plt.show()

In [ ]:
sns.scatterplot(x='CashbackAmount', y='DaySinceLastOrder', data=eda_df)
plt.show()

In [ ]:
# ax = eda_df.query('Tenure != 0 and DaySinceLastOrder != 0') \
ax= eda_df.groupby(['Tenure'])['CashbackAmount'].mean() \
    .head(5) \
    .plot(kind='bar')

ax.set_ylabel('Average Cashback Amount by Tenure')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.show()





## EDA Conclusion

- Những khách hàng lâu năm (thể hiện bởi high Tenure) có tỉ lệ rời bỏ thấp hơn.
- Bên cạnh tỉ lệ giao dịch thường xuyên và cashback amount cũng góp phần giữ khách hàng ở lại và làm giảm tỉ lệ rời bỏ
- Khách hàng thường có xu hướng rời bỏ trong 1 năm đầu tiên. **(Chiếm >65% total churn của dataset)**

--- 
- Summary:
    - Cần tập trung chăm sóc những khách hàng mới để giảm tỉ lệ rời bỏ trong 1 năm đầu
    - Đa đạng hóa các chương trình cashback amount, và các chương trình những chương trình ưu đãi khác như các chương trình kỷ niệm 1 năm tính từ ngày có giao dịch đầu tiên
    - Thường xuyên tương tác và nhắc khách quay lại mua hàng định kỳ. Qua đó giảm tỉ lệ rời bỏ.

---

# Apply Machine Learning Model:

In [ ]:
# columns_trans = make_column_transformer(
#     (OneHotEncoder(), cat_cols), remainder='passthrough')

# columns_trans = make_column_transformer((StandardScaler(), num_cols), remainder='passthrough')

logreg = LogisticRegression(random_state=99)
dtc = DecisionTreeClassifier(max_depth=16, random_state=42)

In [ ]:
pipe = Pipeline(steps=[('classifier', dtc)])

In [ ]:
full_features_model = cross_val_score(pipe, X_train_scaled,
                                      y_train, cv=10, scoring='accuracy',
                                      error_score='raise').mean()
full_features_model

In [ ]:
important_features_model = cross_val_score(pipe, X_train_selected,
                                           y_train, cv=10, scoring='accuracy',
                                           error_score='raise').mean()
important_features_model

## Evaluating classification models

### Full Features Dataset

In [ ]:
models = {"Logistic Regression": LogisticRegression(random_state=42), 
          "KNN": KNeighborsClassifier(),
          "Decision Tree": DecisionTreeClassifier(random_state=42)}
full_features_results = []

for model in models.values():
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kf)
    full_features_results.append(cv_results)

plt.boxplot(full_features_results, labels=models.keys())
plt.show()

In [ ]:
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    test_score = model.score(X_test_scaled, y_test)
    print(f"{name} Test Set Accuracy: {test_score*100:.2f}%")

----


### Important Features Model Results

In [ ]:
models = {"Logistic Regression": LogisticRegression(random_state=42), 
          "KNN": KNeighborsClassifier(),
          "Decision Tree": DecisionTreeClassifier(random_state=42)}
important_features_results = []

for model in models.values():
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    cv_results = cross_val_score(model, X_train_selected, y_train, cv=kf)
    important_features_results.append(cv_results)

plt.boxplot(important_features_results, labels=models.keys())
plt.show()

In [ ]:
for name, model in models.items():
    model.fit(X_train_selected, y_train)
    test_score = model.score(X_test_selected, y_test)
    print(f"{name} Test Set Accuracy: {test_score*100:.2f}%")